In [ ]:
# Copyright: 2020 The Johns Hopkins University Applied Physics Laboratory LLC (JHU/APL).
# All Rights Reserved.
#
# This material may be only be used, modified, or reproduced by or for the U.S.
# Government pursuant to the license rights granted under the clauses at DFARS
# 252.227-7013/7014 or FAR 52.227-14. For any other permission, please contact
# the Office of Technology Transfer at JHU/APL: Telephone: 443-778-2792,
# Email: techtransfer@jhuapl.edu, Website: http://www.jhuapl.edu/ott/
#
# NO WARRANTY, NO LIABILITY. THIS MATERIAL IS PROVIDED "AS IS." JHU/APL MAKES
# NO REPRESENTATION OR WARRANTY WITH RESPECT TO THE PERFORMANCE OF THE
# MATERIALS, INCLUDING THEIR SAFETY, EFFECTIVENESS, OR COMMERCIAL VIABILITY,
# AND DISCLAIMS ALL WARRANTIES IN THE MATERIAL, WHETHER EXPRESS OR IMPLIED,
# INCLUDING (BUT NOT LIMITED TO) ANY AND ALL IMPLIED WARRANTIES OF PERFORMANCE,
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, AND NON-INFRINGEMENT OF
# INTELLECTUAL PROPERTY OR OTHER THIRD PARTY RIGHTS. ANY USER OF THE MATERIAL
# ASSUMES THE ENTIRE RISK AND LIABILITY FOR USING THE MATERIAL. IN NO EVENT
# SHALL JHU/APL BE LIABLE TO ANY USER OF THE MATERIAL FOR ANY ACTUAL, INDIRECT,
# CONSEQUENTIAL, SPECIAL OR OTHER DAMAGES ARISING FROM THE USE OF, OR INABILITY
# TO USE, THE MATERIAL, INCLUDING, BUT NOT LIMITED TO, ANY DAMAGES FOR LOST
# PROFITS.

import sys
sys.path.append('./src')
import mezze
import numpy as np
from MultiAxisDephasingPMD import MultiAxisDephasing
from Gates import Gates
import qecc
import matplotlib.pyplot as plt
%matplotlib inline

#
#
# This script generates the plots for the supplementary material showing the controls and noise in Fig. S3.
#
#

# Parameters that strongly impact runtime. dcv=1000 is typically used. This makes each trotter step 1/1000 of a gate.
# num_realz is the number of monte carlo trials used for both the Trotter and SchWARMA simulations. num_realz=1000
# provides good convergance.
dcv = 1000 #down conversion in time from mezze to schwarma
num_realz = 1 # Number of monte carlo trials for both Trotter and SchWARMA simulation

# Noise spectra parameters to be run
s_pow_list = [1.0e-12, 1.0e-10, 1e-8, 1e-6, 1e-4, 1e-2]
s_pow = s_pow_list[4]
s_amp = np.sqrt(s_pow)
NN_spec = 24 # Number of discrete time steps in the quantum circuit. This is number of SchWARMA steps
corr_time_list = [0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0]
corr_time = corr_time_list[6]

# Other setup parameters
NN = NN_spec * dcv # Total number of steps in the Trotter circuit
T_max = float(NN_spec) # Sets gates to 1 unit of time
dt = T_max / NN
num_qubits = 5

# Setup mezze configuration. This is the full Trotter based simulation of the circuit
config = mezze.SimulationConfig()  # Get the config structure (class)
config.num_steps = NN  # Number of time steps to use in the simulation (larger numbers are more accurate)
config.time_length = T_max  # Set the time length of the integration
config.dt = float(config.time_length) / float(config.num_steps)  # Set the time-step size
config.parallel = False  # Set to true for parallel Monte Carlo, False otherwise
config.num_runs = num_realz  # This sets the number of Monte Carlo trials to use to compute the average superoperator
config.get_unitary = False  # Report unitary as opposed to Liouvillian
time_axis = np.linspace(0., config.time_length, config.num_steps)

# Initialize Physical Machine Description
pmd = MultiAxisDephasing(num_qubits, [s_pow, s_pow, s_pow], [corr_time, corr_time, corr_time])
# Surface Syndome Circuit
surf_xsyndrome = qecc.SurfaceSyndrome()
# Build controls
surf_xsyndrome.generate_xsyndrome_controls()
sys.stdout.flush()

# Build control pulses for the surface code syndrome circuit
circuit = Gates(pmd, config)
sq_ctrls = []
for n in range(num_qubits):
    ctrls = circuit.generate_circuit_controls(surf_xsyndrome.ctrl_list[n])
    for cc in ctrls:
        sq_ctrls.append(cc)
tq_ctrls = []
for n in range(5, len(surf_xsyndrome.ctrl_list)):
    ctrls = circuit.generate_circuit_controls(surf_xsyndrome.ctrl_list[n])
    tq_ctrls.append(ctrls[2])

controls = np.transpose(np.concatenate((sq_ctrls, tq_ctrls), axis=0))


ham_func = mezze.HamiltonianFunction(pmd)
ham = mezze.PrecomputedHamiltonian(pmd, controls, config, ham_func.get_function())
sim = mezze.Simulation(config, pmd, ham)
report = sim.run()


In [ ]:
from matplotlib import rc
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams['text.usetex'] = True
rc('text', usetex=True)
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams.update({'axes.titlesize':20, 'axes.labelsize':18, 'legend.fontsize':12, 'xtick.labelsize':16, 
                     'ytick.labelsize':14,'figure.figsize':(8.4,3.36), })
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

time_axis = np.linspace(0.0, T_max, NN)
plt.plot(time_axis, np.transpose(sim.hamiltonian.controls)[0], label="Qubit 0: X control")
plt.plot(time_axis, np.transpose(sim.hamiltonian.controls)[1], label="Qubit 0: Y control")
plt.plot(time_axis, np.transpose(sim.hamiltonian.controls)[15], label="Qubit (0, 1): ZZ control")
plt.plot(time_axis, np.transpose(sim.hamiltonian.controls)[16], label="Qubit (0, 2): ZZ control")
plt.plot(time_axis, np.transpose(sim.hamiltonian.controls)[17], label="Qubit (0, 3): ZZ control")
plt.plot(time_axis, np.transpose(sim.hamiltonian.controls)[18], label="Qubit (0, 4): ZZ control")
plt.xlim([np.min(time_axis), np.max(time_axis)])
plt.title("X Syndrome Qubit 0 Controls")
plt.xlabel('Time (Gate Lengths)')
plt.ylabel('Amplitude (1/Gate Lengths)')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), ncol=3)
#plt.tight_layout()
plt.savefig("./XSyndromeQubit0Controls.pdf", bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(time_axis, sim.hamiltonian.noise_sources[0].generate(), label="Qubit 0: X noise")
plt.plot(time_axis, sim.hamiltonian.noise_sources[1].generate(), label="Qubit 0: Y noise")
plt.plot(time_axis, sim.hamiltonian.noise_sources[2].generate(), label="Qubit 0: Z noise")
plt.xlim([np.min(time_axis), np.max(time_axis)])
plt.title("X Syndrome Qubit 0 Noise")
plt.xlabel('Time (Gate Lengths)')
plt.ylabel('Amplitude (1/Gate Lengths)')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), ncol=3)
plt.savefig("./XSyndromeQubit0Noise.pdf", bbox_inches='tight')
plt.show()